This notebook follows this course: https://www.kaggle.com/learn/geospatial-analysis


The notes and comments are personal approaches

In [ ]:

import folium
from folium import Marker, GeoJson
from folium.plugins import HeatMap

import pandas as pd
import geopandas as gpd

In [ ]:
releases = gpd.read_file("../input/geospatial-learn-course-data/toxic_release_pennsylvania/toxic_release_pennsylvania/toxic_release_pennsylvania.shp") 
releases.head()

In [ ]:
stations = gpd.read_file("../input/geospatial-learn-course-data/PhillyHealth_Air_Monitoring_Stations/PhillyHealth_Air_Monitoring_Stations/PhillyHealth_Air_Monitoring_Stations.shp")
stations.head()

In [ ]:
#make sure they use the CRS
print(stations.crs)
print(releases.crs)
#yes they are 

In [ ]:
# Select one release incident in particular
recent_release = releases.iloc[360]

# Measure distance from release to each station
distances = stations.geometry.distance(recent_release.geometry)
distances

In [ ]:
print('Mean distance to monitoring stations: {} feet'.format(distances.mean()))


In [ ]:
print('Closest monitoring station ({} feet):'.format(distances.min()))
print(stations.iloc[distances.idxmin()][["ADDRESS", "LATITUDE", "LONGITUDE"]])

In [ ]:
two_mile_buffer = stations.geometry.buffer(2*5280)
two_mile_buffer.head()

In [ ]:
m = folium.Map(location=[39.9526,-75.1652], zoom_start=11)
HeatMap(data=releases[['LATITUDE', 'LONGITUDE']], radius=15).add_to(m)
for idx, row in stations.iterrows():
    Marker([row['LATITUDE'], row['LONGITUDE']]).add_to(m)
    
# Plot each polygon on the map
GeoJson(two_mile_buffer.to_crs(epsg=4326)).add_to(m)

# Show the map
m

In [ ]:
# Turn group of polygons into single multipolygon
my_union = two_mile_buffer.geometry.unary_union
print('Type:', type(my_union))

# Show the MultiPolygon object
my_union

In [ ]:
# The closest station is less than two miles away
my_union.contains(releases.iloc[360].geometry)

In [ ]:
# The closest station is more than two miles away
my_union.contains(releases.iloc[358].geometry)
